<a href="https://colab.research.google.com/github/b62mp1/files/blob/main/Graphs%26Metrics/Optimizers/Loss%20Functions/Binary_CrossEntropy/LSTM_BCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Activation, concatenate, Dropout
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go
import plotly

In [30]:
stock_name = "AMZN"
dataset=pdr.get_data_yahoo(stock_name, start='2020-01-01', end='2022-04-19').reset_index()
dataset.index = dataset['Date']

print("Number of rows and columns:", dataset.shape)
dataset.head(5)

[*********************100%***********************]  1 of 1 completed
Number of rows and columns: (578, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2020-01-02,2020-01-02,1875.000000,1898.010010,1864.150024,1898.010010,1898.010010,4029000
2020-01-03,2020-01-03,1864.500000,1886.199951,1864.500000,1874.969971,1874.969971,3764400
2020-01-06,2020-01-06,1860.000000,1903.689941,1860.000000,1902.880005,1902.880005,4061800
2020-01-07,2020-01-07,1904.500000,1913.890015,1892.040039,1906.859985,1906.859985,4044900
2020-01-08,2020-01-08,1898.040039,1911.000000,1886.439941,1891.969971,1891.969971,3508000


In [31]:
close_data = dataset['Close'].values
close_data = close_data.reshape((-1,1))

scaler = MinMaxScaler(feature_range=(0, 1))
close_data = scaler.fit_transform(close_data)

In [32]:
# Splitting data into training and testing sets
split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = dataset['Date'][:split]
date_test = dataset['Date'][split:]

In [33]:
look_back = 5

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=10)

In [34]:
def predict(num_prediction, model):
    prediction_list = close_test[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = dataset['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

In [35]:
# Building LSTM model
model = Sequential()
model.add(
    LSTM(1000,
        activation='relu',
        input_shape=(look_back,1)
        )
)
model.add(Dense(1))
model.compile(optimizer="adam", loss='binary_crossentropy')

num_epochs = 25
model.fit(train_generator,epochs=num_epochs, verbose=1)

Epoch 1/25
23/23 [==============================] - 5s 152ms/step - loss: 9.7929
Epoch 2/25
23/23 [==============================] - 5s 239ms/step - loss: 9.7929
Epoch 3/25
23/23 [==============================] - 6s 251ms/step - loss: 9.7929
Epoch 4/25
23/23 [==============================] - 7s 300ms/step - loss: 9.7929
Epoch 5/25
23/23 [==============================] - 5s 228ms/step - loss: 9.7929
Epoch 6/25
23/23 [==============================] - 3s 144ms/step - loss: 9.7929
Epoch 7/25
23/23 [==============================] - 3s 144ms/step - loss: 9.7929
Epoch 8/25
23/23 [==============================] - 3s 144ms/step - loss: 9.7929
Epoch 9/25
23/23 [==============================] - 3s 146ms/step - loss: 9.7929
Epoch 10/25
23/23 [==============================] - 3s 143ms/step - loss: 9.7929
Epoch 11/25
23/23 [==============================] - 3s 143ms/step - loss: 9.7929
Epoch 12/25
23/23 [==============================] - 3s 146ms/step - loss: 9.7929
Epoch 13/25
23/23 [======

In [36]:
# Predict and plot the results
prediction = model.predict(test_generator)

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data',
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Predicted Data'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Test Data'
)
layout = go.Layout(
    title = stock_name,
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()